In [121]:
import numpy as np
from typing import Dict
import random
import math

In [122]:
data = np.loadtxt(open("./data.csv", "rb"), delimiter=",", skiprows=1)

In [123]:
X = np.delete(data, 3, 1)
y = data[:, 3] + 10 # y = 5*x1 + 3*x2 + x3 + 10 ( c = 10)
y = y.reshape((y.shape[0], 1))


In [124]:
def rows(mat: np.ndarray) -> int:
    return mat.shape[0]

def cols(mat: np.ndarray) -> int:
    return mat.shape[1]

In [125]:
def linear_regression_forward(X_batch: np.ndarray,
                              y_batch: np.ndarray,
                              weights: np.ndarray,
                              bias: float) -> float:
    assert rows(X_batch) == rows(y_batch), f"{rows(X_batch)} != {rows(y_batch)}"
    assert y_batch.shape[1] == 1, "target vector must have exactly 1 column"
    assert cols(X_batch) == rows(weights), f"{cols(X_batch)} != {rows(weights)}"

    unbiased_prediction = np.dot(X_batch, weights)
    prediction = unbiased_prediction + bias
    loss = np.mean(np.power((prediction - y_batch), 2))

    return loss, prediction, unbiased_prediction

In [126]:
linear_regression_forward(X, y, np.random.rand(cols(X), 1), 1)

(3235.662657683489,
 array([[ 9.17724494],
        [ 8.1705777 ],
        [ 9.4162333 ],
        [15.33130316],
        [11.98351256],
        [16.03543579],
        [21.92227119],
        [ 7.55565623],
        [11.12682784],
        [17.10266908],
        [17.83226127],
        [15.30584359],
        [17.41803614],
        [ 8.44765275],
        [15.92076506],
        [19.29422055],
        [10.17107974],
        [ 6.27191393],
        [ 8.93825658],
        [ 9.16739271],
        [14.96522228],
        [21.45712692],
        [ 9.71876792],
        [11.26675281],
        [11.40390289],
        [16.27442414],
        [19.54583603],
        [14.00947417],
        [ 6.45998315],
        [10.16122751],
        [16.34782264],
        [16.21067256],
        [ 3.45301919],
        [ 4.22069808],
        [16.71390352],
        [14.8280722 ],
        [12.39773769],
        [22.71540964],
        [17.31917808],
        [15.61823044],
        [20.46329212],
        [13.84408963],
        [ 7.07

In [127]:
def lg_train(X: np.ndarray, y: np.ndarray, learn_rate: float = 0.01, batch_size: int = 10, iters: int = 20):
    """Train the linear regression model on a dataset with samples `X` and target `y`"""

    assert X.ndim == 2 and y.ndim == 2, f"{X.ndim} {y.ndim}"
    assert y.shape[1] == 1, "target vector must have exactly 1 column"
    assert rows(X) == rows(y), "dimensions of samples and targets do not match"

    def init_weights() -> np.ndarray:
        """Initialize a random weight vector"""
        num_weights = cols(X)
        return np.random.randn(num_weights, 1)

    def loss_gradients(batch: Dict[str, np.ndarray],
                        weights: np.ndarray,
                        prediction: np.ndarray,
                        unbiased_prediction: np.ndarray):
        X_, y_ = batch['X'], batch['y']
        
        grad_pred__weights  = np.transpose(X_, (1, 0))
        grad_loss__pred_fn = 2 * (prediction - y_)
        
        return np.dot(grad_pred__weights, grad_loss__pred_fn)

    weights = init_weights()
    bias = random.randint(0, 10)
    prev_loss = float('inf')
    for i in range(iters):
        # In each iteration, we extract some random sample data and their corresponding
        # targets and group it as one 'batch'.
        batch: Dict[str, np.ndarray] = dict()
        start_index = random.randint(0, rows(X) - batch_size - 1)
        end_index = start_index + batch_size
        batch['X'] = X[start_index: end_index]
        batch['y'] = y[start_index: end_index]

        loss, pred, unbiased_pred = \
            linear_regression_forward(
                batch['X'],
                batch['y'],
                weights,
                bias
            )

        grad = loss_gradients(batch, weights, pred, unbiased_pred)
        weights -= learn_rate * grad

    return weights, bias

In [129]:
def lg_predict(train_X, train_y, test_X, test_y):
  weights, bias = lg_train(train_X, train_y)
  pred = np.dot(test_X, weights) + bias
  for i in range(test_y.shape[0]):
    print(pred[i] + bias, test_y[i])

train_size = math.floor(0.8 * X.shape[0])
test_size = X.shape[0] - train_size

lg_predict(X[:train_size], y[:train_size], X[train_size:], y[train_size:])

[[-2.02775135e+30]
 [-4.44831048e+30]
 [-1.07112458e+30]]
[-7.9398471e+31] [75.]
[-6.85885965e+31] [61.]
[-5.64786017e+31] [52.]
[-1.7522131e+31] [45.]
[-6.84740986e+31] [64.]
[-4.21842426e+31] [58.]
[-4.44831048e+30] [13.]
[-8.38467814e+31] [78.]
[-1.01647222e+32] [109.]
[-4.43758061e+31] [62.]
[-1.4416056e+31] [20.]
[-4.33046815e+31] [61.]
[-7.91766744e+31] [100.]
[-6.09341113e+31] [74.]
[-1.8078751e+31] [37.]
[-4.59542363e+31] [53.]
[-2.3883695e+31] [56.]
[-2.22415524e+31] [25.]
[-9.83773353e+31] [85.]
[-7.73779187e+31] [89.]
[-9.79845275e+31] [92.]
[-2.75391909e+31] [54.]
[-9.23505946e+31] [91.]
[-2.88321121e+31] [30.]
[-9.8039334e+30] [18.]
[-6.16124281e+31] [82.]
[-8.58817318e+31] [102.]
[-4.67963652e+31] [60.]
[-1.07444967e+32] [109.]
[-1.93208867e+31] [56.]
[-5.89063599e+31] [69.]
[-4.85386075e+31] [50.]
[-4.24625525e+31] [54.]
[-4.78674898e+31] [61.]
[-5.41725404e+31] [51.]
[-1.33521305e+31] [38.]
[-8.07479055e+31] [72.]
[-6.13269191e+31] [67.]
[-9.75917197e+31] [99.]
[-2.8046